# Лекция 5: Форматы данных (часть 2)

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2020 г. 

При подготовке лекции использованы материалы:
* Документация к рассмотренным пакетам

V 0.2 04.10.2021

## Разделы: <a class="anchor" id="разделы"></a>
* [Хранение табличных данных](#хранение)
* [CSV](#csv)
* [NPY](#npy)
* [HDF](#hdf)
* [Apache Parquet](#parquet)
-
* [к оглавлению](#разделы)

In [87]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v1.css")
HTML(html.read().decode('utf-8'))

# Хранение табличных данных <a class="anchor" id="хранение"></a>
* [к оглавлению](#разделы)

__Разнообразие форматов хранения табличных данных__

<center>         
    <img src="./img/file_perfomance.png" alt="Сравнение производительности" style="width: 500px;"/>
    <b>Сравнение производительности при работе с различными форматами файлов</b>
</center>

Сравнение:
* Plain-text CSV — a good old friend of a data scientist
* Pickle — a Python’s way to serialize things
* MessagePack — it’s like JSON but fast and small
* HDF5 — a file format designed to store and organize large amounts of data
* Feather — a fast, lightweight, and easy-to-use binary file format for storing data frames
* Parquet — an Apache Hadoop’s columnar storage format

Резульаты сравнения: https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

__Сложности представления иерархических данных в табличном виде__

Пример описания в профессиональной соцсети в формате JSON:

```JSON
{
  "user_id":     251,
  "first_name":  "Bill",
  "last_name":   "Gates",
  "summary":     "Co-chair of the Bill & Melinda Gates... Active blogger.",
  "region_id":   "us:91",
  "industry_id": 131,
  "photo_url":   "/p/7/000/253/05b/308dd6e.jpg",
  "positions": [
    {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
    {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
  ],
  "education": [
    {"school_name": "Harvard University",       "start": 1973, "end": 1975},
    {"school_name": "Lakeside School, Seattle", "start": null, "end": null}
  ],
  "contact_info": {
    "blog":    "http://thegatesnotes.com",
    "twitter": "http://twitter.com/BillGates"
  }
}
```

<center>         
    <img src="./img/json_struct.png" alt="Иерархическая структура документа" style="width: 500px;"/>
    <b>Иерархическая структура документа</b>
</center>

<center>         
    <img src="./img/json_compare.png" alt="Сравнение представления документа и реляционной модели" style="width: 650px;"/>
    <b>Сравнение представления документа и реляционной модели</b>
</center>

<center>         
    <img src="./img/translation_solution.png" alt="Вариант решения проблемы представления иерархических данных в табличном виде" style="width: 500px;"/>
    <b>Вариант решения проблемы представления иерархических данных в табличном виде</b>
</center>

Другая потенциальная проблема:
* Динамическая схема документа.

CSV, numpy-native, parket, xlsx, sqlite

# CSV <a class="anchor" id="csv"></a>
* [к оглавлению](#разделы)

__CSV__ (Comma-Separated Values — значения, разделённые запятыми) — текстовый формат, предназначенный для представления табличных данных. Строка таблицы соответствует строке текста, которая содержит одно или несколько полей, разделенных запятыми.

* Формат CSV стандартизирован не полностью. 
* Ключевые проблема: в табличных данных могут иметься запятые или переводы строк
* Популярным решением проблемы запятых и переносов строк является __заключение данных в кавычки__, однако исходные данные могут содержать кавычки. 
* Термином `CSV` могут обозначаться похожие форматы, в которых разделителем является символ табуляции (`TSV`) или точка с запятой.
* Многие приложения, которые работают с форматом CSV, позволяют выбирать символ разделителя и символ кавычек.


Спецификация:
* Каждая __строка файла__ — это одна строка таблицы.
* __Разделителем__ (delimiter) значений колонок является символ запятой `,`. Однако на практике часто используются другие разделители, то есть формат путают с DSVruen и TSV (см. ниже).
* Значения, содержащие зарезервированные символы (двойная кавычка, запятая, точка с запятой, новая строка) обрамляются двойными кавычками (`"`). Если в значении встречаются кавычки — они представляются в файле в виде двух кавычек подряд (`""`).

__Примеры__

Пример 1:

```
1997,Ford,E350,"ac, abs, moon",3000.00
1999,Chevy,"Venture «Extended Edition»","",4900.00
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.00
```
<center>         
    <img src="./img/csv1.png" alt="Иерархическая структура документа" style="width: 500px;"/>
    <b>Результат обработки примера 1</b>
</center>

Пример 2:

```
1965;Пиксель;E240 – формальдегид (опасный консервант)!;"красный, зелёный, битый";"3000,00"
1965;Мышка;"А правильней использовать ""Ёлочки""";;"4900,00"
"Н/д";Кнопка;Сочетания клавиш;"MUST USE! Ctrl, Alt, Shift";"4799,00"
```

<center>         
    <img src="./img/csv2.png" alt="Иерархическая структура документа" style="width: 500px;"/>
    <b>Результат обработки примера 2</b>
</center>

In [88]:
import csv
import pandas as pd

In [89]:
df0 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},
                  index=['a', 'b', 'c'])

df0

,A,B
a,1,4
b,2,5
c,3,6


```python
csv.reader(csvfile, dialect='excel', **fmtparams)
```
* fmtparams : https://docs.python.org/3/library/csv.html#csv-fmt-params

In [90]:
with open('participants_.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    header = next(csv_reader)      
    print(f'Заголовок: {"| ".join(header)}')
    for line_count, row in enumerate(csv_reader, 1):
        print(f'Стока {line_count}: {"| ".join(row)}')          
    print(f'Обработано {line_count} строк.')

Заголовок: Last Name| First Name| Company Name| Company Department| Assigned Classifications| City| State| Country
Стока 1: AALTONEN| Wanida| University| Graduate student| School or university (student)| London| | United Kingdom
Стока 2: ABDELHAMID| Mohamed| alexandria university| student| School or university (student)| alexandria| | Egypt
Стока 3: ABDELMEGUID| Sheref| IBM EGYPT BRANCH| Software Engineer| School or university (staff)| Alexandria| Esatern| Egypt
Стока 4: ABDOU| Lahadji| TIFAKI HAZI| MANAGER| Association| MAMOUDZOU| | France
Стока 5: ABEJIDE| Oluwaseun Adeyemi| Individual| Project Officer| National government (members + staff)| Ido-Osi| Ekiti State| Nigeria
Стока 6: ABTAHIFOROUSHANI| Seyedehasieh| Student| Student| School or university (student)| ROMAINVILLE| | France
Стока 7: ABUELALA| Sherif| AGAP company| Architect| Other company| Alexandria| | Egypt
Стока 8: ABUNEMEH| Omar| omar kamal| social worker| Association| tulkarem| west bank| Palestinian Territory, Occupied


Стока 1442: MURA| Salvatore| Cooperativa Sociale Progetto H| Amministratore| Cooperative| Cagliari| | Italy
Стока 1443: MURAVJOVA| Olga| European Commission| Internal Consultant| European Commission| Brussels| | Belgium
Стока 1444: MURPHY| Geoffrey| scic (eu com)| freelance EN interpreter| European Commission| paris| | France
Стока 1445: MUSENGA| Louange| ?tudiante| ?tudiante| School or university (student)| Strasbourg| | France
Стока 1446: NAETT| Caroline| Coop FR| Secretary General| Cooperative| PARIS| | France
Стока 1447: NAGUIB| Barbara| Student| Politican| School or university (student)| Wuppertal| NRW| Germany
Стока 1448: NAGUIB| Mohamed| Egyptian| accountant| Social enterprise limited by shares| Alexandria| egypt| Egypt
Стока 1449: NAILLON| Delphine| Communaut? urbaine de Strasbourg| Promotion de l'Entrepreneriat| Local authority (members + staff)| Strasbourg| | France
Стока 1450: NAKOS| Spyros| EDIC Komotini| social sientist| Social enterprise limited by shares| komotini| | Gre

Стока 2079: VAN ECHELPOEL| Koen| Triodos Bank| Go to market manager Business| Financial institution| Brussels| | Belgium
Стока 2080: VAN EGTEN| Jacques| CESES| President| Association| brussels| | Belgium
Стока 2081: VAN GEERSTOM| Julien| Pr?sident PPS Social Integration, anti-Poverty Policy, Social Economy and Federal Urban Policy| Pr?sident PPS Social Integration, anti-Poverty Policy, Social Economy and Federal Urban Policy| National government (members + staff)| Brussels| | Belgium
Стока 2082: VAN HOOIJDONK| Imke| European Economic and Social Committee| Assistant| European Economic and Social Committee (staff)| Bruxelles| | Belgium
Стока 2083: VANDEN EYNDE| Olivier| Close the Gap International vzw| Director| Foundation| Brussel| | Belgium
Стока 2084: VANDERVELDEN| Charles| La Ruelle asbl| Executive director| Association| St-Josse-Ten-Noode| Bruxelles capitale| Belgium
Стока 2085: VANDEWINCKELE| An-Rose| Levanto| managing director| Other company| Deurne| Antwerp| Belgium
Стока 2086: V

Прямое чтение CSV в словарь

```python
class csv.DictReader(f, fieldnames=None, restkey=None, restval=None, dialect='excel', *args, **kwds)
```

The `fieldnames` parameter is a sequence.
* If fieldnames is omitted, the values in the first row of file f will be used as the fieldnames. 
* Regardless of how the fieldnames are determined, the dictionary preserves their original ordering.
* If a row has more fields than fieldnames, the remaining data is put in a list and stored with the fieldname specified by restkey (which defaults to `None`).
* If a non-blank row has fewer fields than fieldnames, the missing values are filled-in with the value of restval (which defaults to `None`).

In [91]:
with open('participants_.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=';')
    header = next(csv_reader)      
    print(f'Заголовок: {"| ".join(header)}')
    
    for line_count, row in enumerate(csv_reader, 1):
#         Last Name| First Name| Company Name
        print(f'\t{row["First Name"]} {row["Last Name"]} работает в {row["Company Name"]}.')
    print(f'Обработано {line_count} строк.')

Заголовок: Last Name| First Name| Company Name| Company Department| Assigned Classifications| City| State| Country
	Mohamed ABDELHAMID работает в alexandria university.
	Sheref ABDELMEGUID работает в IBM EGYPT BRANCH.
	Lahadji ABDOU работает в TIFAKI HAZI.
	Oluwaseun Adeyemi ABEJIDE работает в Individual.
	Seyedehasieh ABTAHIFOROUSHANI работает в Student.
	Sherif ABUELALA работает в AGAP company.
	Omar ABUNEMEH работает в omar kamal.
	Arjun Bahadur ACHARYA работает в Wave Express Youth Club.
	Sarah ACHERMANN работает в cewas.
	Anne-Laure ACKERMANN работает в ADAPEI du BAS RHIN.
	Raoul ACKERMANN работает в -.
	Jose ACOSTA VALDEZ работает в 012-0100922-0.
	Filippo ADDARII работает в Young Foundation.
	Michael ADESANWO работает в T..
	Shree Ram ADHIKARI работает в N/A.
	Livia ADINOLFI работает в Une.
	J?romine ADLER работает в Sciences Po Strasbourg.
	Yawo Nevemde ADOSSI работает в UNIVERSITE UCAO.
	Rudy AERNOUDT работает в Comit? ?conomique et social europ?en.
	Edmund Smith AFRIFA работа

	Lemlah MOHAMMED работает в Jugend Werkstatt Frohe Zukunft.
	Valentin MOLINA работает в Universidad de Granada.
	Inger Steen M?LLER работает в social enterprise.
	Ciara MOLLOY работает в Department of Education and Skills, Ireland.
	Marina MONACO работает в EUROPEAN TRADE UNION CONFEDERATION.
	Jose MONCADA работает в European Securities and Markets Authority.
	Sergio MONDELLO работает в Consorzio Sol.Co. Rete di Imprese Sociali Siciliane.
	Italo MONFREDINI работает в Gruppo Cooperativo CGM.
	Attila Mong работает в Ashoka.
	Assiba MONSSI работает в PARLEMENT AFRICAIN DE LA SOCIETE CIVIL.
	Michel MONTAGU работает в Randstad.
	Jean-Yves MONTARGERON работает в ALSACE  ACTIVE.
	Maureen MONTGOMERY работает в Action with Communities in Rural Kent.
	Raquel MORA работает в INQUVE BUSINESS DEVELOPMENT.
	Leandro MORAIS работает в PUCCAMPINAS; FACAMP; UNICAMP.
	Sergi MORALES DIAZ работает в ENSIE.
	Letizia MORATTI работает в San Patrignano Foundation.
	Giuseppe MORELLI работает в public organizati

	Gavino SOGGIA работает в Gruppo Cooperativo CGM.
	Aldo SOLDI работает в Coopfond.
	Carmen SOLI?O работает в SCIC.
	Marine SOLOMONISHVILI работает в International Foundation LEA.
	Marta SOL?RZANO-GARC?A работает в UNED.
	Fabienne SOMMACAL работает в Mutualit? Fran?aise Lorraine.
	Johannes SOMMERFELD работает в World Health Organization, Special Programme for Research and Training in Tropical Diseases (TDR).
	Egle SONGAILIENE работает в LCC International University.
	Thorkil SONNE работает в Specialist People Foundation.
	Shraban Kumar SOP работает в HOME AND LIFE FOUNDATION (NGO).
	Ivanka SOTIROVA работает в elue.
	Ivanka SOTIROVA работает в Municipality of Stara Zagora.
	Isabel SOTO работает в FAECTA.
	Iv?n SOTO работает в Madrid Emprende (Town Council of Madrid).
	Alexander SOTTSAS работает в career moves.
	Bertrand SOUQUET работает в MGEN.
	Parthenopi SOURMAIDOU работает в Ergani center.
	Nedim S?ZEN работает в Ankara Development Agency.
	Ottavia SPAGGIARI работает в Vita.
	Margarit

Чтение CSV в Pandas:
* Документация: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [4]:
df = pd.read_csv('participants_.csv', delimiter=';')
df[:10]

,Last Name,First Name,Company Name,Company Department,Assigned Classifications,City,State,Country
0,AALTONEN,Wanida,University,Graduate student,School or university (student),London,NaN,United Kingdom
1,ABDELHAMID,Mohamed,alexandria university,student,School or university (student),alexandria,NaN,Egypt
2,ABDELMEGUID,Sheref,IBM EGYPT BRANCH,Software Engineer,School or university (staff),Alexandria,Esatern,Egypt
3,ABDOU,Lahadji,TIFAKI HAZI,MANAGER,Association,MAMOUDZOU,NaN,France
4,ABEJIDE,Oluwaseun Adeyemi,Individual,Project Officer,National government (members + staff),Ido-Osi,Ekiti State,Nigeria
5,ABTAHIFOROUSHANI,Seyedehasieh,Student,Student,School or university (student),ROMAINVILLE,NaN,France
6,ABUELALA,Sherif,AGAP company,Architect,Other company,Alexandria,NaN,Egypt
7,ABUNEMEH,Omar,omar kamal,social worker,Association,tulkarem,west bank,"Palestinian Territory, Occupied"
8,ACHARYA,Arjun Bahadur,Wave Express Youth Club,Program Officer,Other,Kathmandu,NaN,Nepal
9,ACHERMANN,Sarah,cewas,Project Manager,Association,Willisau,NaN,Switzerland


__Запись в CSV__

In [5]:
with open('employee_file.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', 
                                 quotechar='"', quoting=csv.QUOTE_MINIMAL)

    employee_writer.writerow(['John Smith', 'Accounting', 'November'])
    employee_writer.writerow(['Erica Meyers', 'IT', 'March'])

In [6]:
with open('employee_file.csv', mode='r') as employee_file:
    for line in employee_file:
        print(line, end="")

John Smith,Accounting,November

Erica Meyers,IT,March



In [86]:
with open('employee_file2.csv', mode='w') as csv_file:
    fieldnames = ['emp_name', 'dept', 'birth_month']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'emp_name': 'John Smith', 'dept': 'Accounting', 'birth_month': 'November'})
    writer.writerow({'emp_name': 'Erica Meyers', 'dept': 'IT', 'birth_month': 'March'})

In [87]:
with open('employee_file2.csv', mode='r') as employee_file:
    for line in employee_file:
        print(line, end="")

emp_name,dept,birth_month

John Smith,Accounting,November

Erica Meyers,IT,March



Работа с CSV в Pandas:
Документация:
* чтение: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
* запись: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv

In [88]:
participants_df = pd.read_csv('participants_.csv', delimiter=';')
participants_df[:10]

,Last Name,First Name,Company Name,Company Department,Assigned Classifications,City,State,Country
0,AALTONEN,Wanida,University,Graduate student,School or university (student),London,NaN,United Kingdom
1,ABDELHAMID,Mohamed,alexandria university,student,School or university (student),alexandria,NaN,Egypt
2,ABDELMEGUID,Sheref,IBM EGYPT BRANCH,Software Engineer,School or university (staff),Alexandria,Esatern,Egypt
3,ABDOU,Lahadji,TIFAKI HAZI,MANAGER,Association,MAMOUDZOU,NaN,France
4,ABEJIDE,Oluwaseun Adeyemi,Individual,Project Officer,National government (members + staff),Ido-Osi,Ekiti State,Nigeria
5,ABTAHIFOROUSHANI,Seyedehasieh,Student,Student,School or university (student),ROMAINVILLE,NaN,France
6,ABUELALA,Sherif,AGAP company,Architect,Other company,Alexandria,NaN,Egypt
7,ABUNEMEH,Omar,omar kamal,social worker,Association,tulkarem,west bank,"Palestinian Territory, Occupied"
8,ACHARYA,Arjun Bahadur,Wave Express Youth Club,Program Officer,Other,Kathmandu,NaN,Nepal
9,ACHERMANN,Sarah,cewas,Project Manager,Association,Willisau,NaN,Switzerland


In [89]:
hrdata_df = pd.read_csv('hrdata.csv')
hrdata_df

,Name,Hire Date,Salary,Sick Days remaining
0,Graham Chapman,03/15/14,50000.0,10
1,John Cleese,06/01/15,65000.0,8
2,Eric Idle,05/12/14,45000.0,10
3,Terry Jones,11/01/13,70000.0,3
4,Terry Gilliam,08/12/14,48000.0,7
5,Michael Palin,05/23/13,66000.0,8


In [90]:
hrdata_df = pd.read_csv('hrdata.csv', index_col='Name')
print(hrdata_df)

               Hire Date   Salary  Sick Days remaining
Name                                                  
Graham Chapman  03/15/14  50000.0                   10
John Cleese     06/01/15  65000.0                    8
Eric Idle       05/12/14  45000.0                   10
Terry Jones     11/01/13  70000.0                    3
Terry Gilliam   08/12/14  48000.0                    7
Michael Palin   05/23/13  66000.0                    8


In [91]:
print(type(hrdata_df['Hire Date'][0]))

<class 'str'>


In [93]:
hrdata_df = pd.read_csv('hrdata.csv', 
            index_col='Emp', 
            parse_dates=['Hired'], 
            header=0, 
            names=['Emp', 'Hired','Salary', 'Sick Days'])

hrdata_df

,Hired,Salary,Sick Days
Emp,,,
Graham Chapman,2014-03-15,50000.0,10
John Cleese,2015-06-01,65000.0,8
Eric Idle,2014-05-12,45000.0,10
Terry Jones,2013-11-01,70000.0,3
Terry Gilliam,2014-08-12,48000.0,7
Michael Palin,2013-05-23,66000.0,8


# Формат для ndarray - npy <a class="anchor" id="npy"></a>

* [к оглавлению](#разделы)

__Формат npy__ - двоичный формат для сохранения (сериализации) произвольных единичных массивов ndarray на диск.
* Формат хранит информацию о:
    * форме (shape)
    * типе элементов (dtype)
* Формат позволяет корректно восстановить массив на компьютере с другой архитектурой.
* Формат задуман как максимально простой, достигающий поставленной цели.
* Поддержка работы с форматом распространяется как часть основного пакета numpy.
* Реализация предназначена для чистого Python и распространяется как часть основного пакета numpy.    

__Сравнение с альтернативными подходами__

Сравнение с использованием pickle:
* Использование pickle приводит к дублированию данных в памяти. Для очень больших массовов это может быть очень затратно или неприемлемо из-за нехватки памяти на компьютере.
* `npy` использует механизм memory-mapping который позволяет не загружать большой объем данных тогда, когда требуется лишь небольшой фрагмент данных.

In [92]:
import numpy as np
import pandas as pd
import pickle

from sys import getsizeof
import os

In [93]:
a1 = np.arange(100)
a2 = np.arange(10000, step=10)

In [94]:
getsizeof(a1)

496

```python
numpy.save(file, arr, allow_pickle=True, fix_imports=True)
```
__Save an array to a binary file in NumPy .npy format.__

__Parameters__

* _file : file, str, or pathlib.Path_

Файл или имя файла, в котором сохраняются данные. Если файл является объектом-файлом, то имя файла не изменяется. Если файл представляет собой строку или путь, к имени файла будет добавлено расширение `.npy`, если этого не было сделано ранее.    
    
* _arr : array_like_

Массив для сохранения

* _allow_pickle : bool, optional_

Разрешить сохранение массивов объектов с помощью Python pickles. Причины запрета на использование pickle включают безопасность (загрузка данных в формате pickle может выполнять произвольный код) и переносимость (объекты pickle могут не загружаться на разных инсталляциях Python, например, если для сохраненных объектов требуются библиотеки, которые недоступны, и не все данные совместимы при переносе между Python 2 и Python 3). __По умолчанию: True__

* _fix_imports : bool, optional_

(Необходим для совместимости между Python 3 и Python 2)

In [95]:
with open('a1.npy', 'wb') as f:
    np.save(f, a1)

In [96]:
os.path.getsize('a1.npy')

528

In [97]:
with open('a1.pickle', 'wb') as f:
    pickle.dump(a1, f)

In [98]:
os.path.getsize('a1.pickle')

558

In [100]:
# загрузка ndarray из файла npy:
with open('a1.npy', 'rb') as f:
    a1_l = np.load(f)

In [101]:
a1_l

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [102]:
with open('a1_2.npy', 'wb') as f:
    np.save(f, a1)
    np.save(f, a2)    

In [103]:
os.path.getsize('a1_2.npy')

4656

In [104]:
with open('a1_2.npy', 'rb') as f:
    a1_l = np.load(f)
    a2_l = np.load(f)    

In [105]:
len(a1_l), len(a2_l)

(100, 1000)

In [106]:
a2_l[:10], a2_l[-10:]

(array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90]),
 array([9900, 9910, 9920, 9930, 9940, 9950, 9960, 9970, 9980, 9990]))

__Использование формата npz__

Формат `.npz` эти архив zip с файлами имеющими имена переменных, которые в нем сохранены. Файл не сжаты и каждый файл в нем это файл содержащий ndarray в формате `.npz`. При открытии файла `.npz` возвращается объект типа `NpzFile`. Это объект имеющий интерфейс словаря из которого можно получить список массивов и содержимое самих массивов. При сохранение ndarray необходимо использовать имена, которые могут быть корректными именами файлов (в т.ч. не содержат `/` или `..`).

```python
numpy.savez(file, *args, **kwds)
```
__Save several arrays into a single file in uncompressed `.npz` format.__

If arguments are passed in with no keywords, the corresponding variable names, in the .npz file, are ‘arr_0’, ‘arr_1’, etc. If keyword arguments are given, the corresponding variable names, in the .npz file will match the keyword names.

__Parameters__
* file : str or file

Either the filename (string) or an open file (file-like object)
        where the data will be saved. If file is a string or a Path, the
        ``.npz`` extension will be appended to the filename if it is not
        already there.
* args : Arguments, optional

Arrays to save to the file. Since it is not possible for Python to
        know the names of the arrays outside `savez`, the arrays will be saved
        with names "arr_0", "arr_1", and so on. These arguments can be any
        expression.
* kwds : Keyword arguments, optional

Arrays to save to the file. Arrays will be saved in the file with the
        keyword names.

In [108]:
np.savez('a.npz', a1, a2)

In [109]:
npzfile = np.load('a.npz')

In [111]:
list(npzfile)

['arr_0', 'arr_1']

In [112]:
npzfile['arr_0']

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [113]:
arrs = list(npzfile.values())
len(arrs)

2

In [115]:
arrs[0][-10:], arrs[1][-10:]

(array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([9900, 9910, 9920, 9930, 9940, 9950, 9960, 9970, 9980, 9990]))

In [116]:
# NpzFile нужно закрывать:
npzfile.close()

In [117]:
npzfile['arr_0']

AttributeError: 'NoneType' object has no attribute 'open'

__Альтернативные способы работы с `.npz`:__

In [29]:
M = 1000_000
S = 10
sz = S * M 
a3 = np.arange(sz)

In [118]:
# сохранение в npz массивов с заданными именами:
np.savez('a_new.npz', a1=a1, a3=a3)

In [119]:
size = os.path.getsize('a_new.npz')
print(f'uncompressed size: {size:,}')

uncompressed size: 40,000,854


In [121]:
# другой стиль чтения из с npz:
with np.load('a_new.npz') as data:
    a3_l = data['a3']
    
a2_l[-10:]

array([9900, 9910, 9920, 9930, 9940, 9950, 9960, 9970, 9980, 9990])

In [122]:
# np.savez('a_compr.npz', a1=a1, a2=a2)

# сохранение в сжатый файл .npz "_compressed":
np.savez_compressed('a_compr.npz', a1=a1, a3=a3)

In [35]:
size = os.path.getsize('a_compr.npz')
print(f'compressed size: {size:,}')

compressed size: 13,831,061


In [124]:
# чтение из сжатого файла не отличается:
with np.load('a_compr.npz') as data:
    a2_l = data['a1']
    
a2_l[-10:]

array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

__Работа с использованием mmap__

* https://numpy.org/doc/stable/reference/generated/numpy.load.html#numpy.load
* https://numpy.org/doc/stable/reference/generated/numpy.memmap.html#numpy.memmap

__Виртуальная память__ (virtual memory) — метод управления памятью компьютера, позволяющий выполнять программы, требующие больше оперативной памяти, чем имеется в компьютере, путём автоматического перемещения частей программы между основной памятью и вторичным хранилищем (например, жёстким диском). 

* Для программы __применение виртуальной памяти полностью прозрачно__ и не требует дополнительных усилий со стороны программиста.
* __Реализация__ виртуальной памяти требует как __аппаратной поддержки__, так и __поддержки со стороны операционной системы__.

<center>         
    <img src="./img/vm1.png" alt="Концепция виртуальной памяти" style="width: 500px;"/>
    <b>Концепция виртуальной памяти</b>
</center>

__Таблица страниц__ (page table)- это структура данных, используемая системой виртуальной памяти в операционной системе компьютера для хранения сопоставления между виртуальным адресом и физическим адресом. Виртуальные адреса используются выполняющимся процессом, в то время как физические адреса используются аппаратным обеспечением, или, более конкретно, подсистемой ОЗУ. Таблица страниц является ключевым компонентом преобразования виртуальных адресов, который необходим для доступа к данным в памяти.

<center>         
    <img src="./img/vm2.png" alt="Концепция виртуальной памяти" style="width: 400px;"/>
    <b>Принцип работы с таблицей страниц</b>
</center>

<center>         
    <img src="./img/mmf.png" alt="Memory-mapped files" style="width: 400px;"/>
    <b>Memory-mapped files</b>
</center>


Модуль Python `mmap` предоставляет возможность работать вводом-выводом (I/O) с помощью memory-mapped file. Это позволяет использовать приемущества низкоуровневых возможностей операционных систем для представления файла в виде одной болшой строки или массива. Это может дать значительный прирост производительности для кода, требующего большого объема ввода-вывода.

* Подробнее про memmap https://docs.python.org/3.0/library/mmap.html , https://realpython.com/python-mmap

In [125]:
M = 1000000
size = 100 * M # 100 M
# size = S * M # 10 M
a_big_1 = np.arange(size)

In [126]:
size = getsizeof(a_big_1)
print(f'size of a_big_1 in memory: {size:,}')

size of a_big_1 in memory: 400,000,096


Пишем и читаем фрагмент с помощью pickle:

In [127]:
with open('a_big_1.pickle', 'wb') as f:
    pickle.dump(a_big_1, f)

In [128]:
size = os.path.getsize('a_big_1.pickle')

print(f'a_big_1.pickle size: {size:,}')

a_big_1.pickle size: 400,000,161


In [129]:
%%time
with open('a_big_1.pickle', 'rb') as f:
    a_big_1_lp = pickle.load(f)

# чтение из середины массива:    
my_data = a_big_1_lp[5 * M : 5 * M + 10000]
my_data[:10], my_data[-10:]

Wall time: 354 ms


(array([5000000, 5000001, 5000002, 5000003, 5000004, 5000005, 5000006,
        5000007, 5000008, 5000009]),
 array([5009990, 5009991, 5009992, 5009993, 5009994, 5009995, 5009996,
        5009997, 5009998, 5009999]))

In [130]:
with open('a_big_1.npy', 'wb') as f:
    np.save(f, a_big_1)

In [131]:
size = os.path.getsize('a_big_1.npy')

print(f'a_big_1.npy: {size:,}')

a_big_1.npy: 400,000,128


__mmap_mode__ = `{None, ‘r+’, ‘r’, ‘w+’, ‘c’}, optional`

Если параметр `memory-map` установлен не в `None`, то используется выбранный режим. 
* Массив с отображением памяти хранится на диске
* но к нему можно получить доступ и выполнить срезы как для любого массива ndarray
* использование mmap особенно полезно для доступа к небольшим фрагментам больших файлов без чтения всего файла в память.

Подробнее о работе с mmap: https://numpy.org/doc/stable/reference/generated/numpy.memmap.html

In [132]:
%%time
a_big_mmap = np.load('a_big_1.npy', mmap_mode='r')

# чтение из середины массива:
my_data = a_big_mmap[5 * M : 5 * M  + 10000]
my_data[:10], my_data[-10:]

Wall time: 1 ms


(memmap([5000000, 5000001, 5000002, 5000003, 5000004, 5000005, 5000006,
         5000007, 5000008, 5000009]),
 memmap([5009990, 5009991, 5009992, 5009993, 5009994, 5009995, 5009996,
         5009997, 5009998, 5009999]))

In [133]:
type(a_big_mmap), type(my_data)

(numpy.memmap, numpy.memmap)

In [134]:
# если необходимо скопировать данные из memmap в обычный ndarray:
my_data_c = np.copy(my_data)
my_data_c[:10], my_data_c[-10:]

(array([5000000, 5000001, 5000002, 5000003, 5000004, 5000005, 5000006,
        5000007, 5000008, 5000009]),
 array([5009990, 5009991, 5009992, 5009993, 5009994, 5009995, 5009996,
        5009997, 5009998, 5009999]))

In [135]:
np.info(my_data)

class:  memmap
shape:  (10000,)
strides:  (4,)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x1d281312d80
byteorder:  little
byteswap:  False
type: int32


In [80]:
np.version.version

'1.18.1'

In [136]:
my_data._mmap

In [137]:
np.info(my_data_c)

class:  ndarray
shape:  (10000,)
strides:  (4,)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x1d2e9486ce0
byteorder:  little
byteswap:  False
type: int32


In [138]:
my_data_c._mmap

AttributeError: 'numpy.ndarray' object has no attribute '_mmap'

Закрытие файла mmap произойдет либо после того, как сборщик мусора удалит все объекты ссылающиеся на файл. Это можно ускорить удалив объекты:
```python
del a_big_mmap
del my_data
```
или явно:
```python
a_big_mmap._mmap.close()
```
но тогда любое обращение использование этого mmap (в том числе и в других объектах) приведет к ошибке. Чтобы избежать проблем, перед закрытием скопируйте необходимые данные в другие ndarray.

In [139]:
a_big_mmap._mmap.close()

In [51]:
# ошибка: mmap уже закрыт (НЕ ВЫПОЛНЯТЬ!)
# my_data2 = a_big_mmap[50 * M : 50 * M + 10000]
# my_data2[:10]

In [140]:
# со скопированными данными все впорядке:
my_data_c[:10], my_data_c[-10:]

(array([5000000, 5000001, 5000002, 5000003, 5000004, 5000005, 5000006,
        5000007, 5000008, 5000009]),
 array([5009990, 5009991, 5009992, 5009993, 5009994, 5009995, 5009996,
        5009997, 5009998, 5009999]))

# Формат HDF <a class="anchor" id="hdf"></a>
* [к оглавлению](#разделы)

__Hierarchical Data Format, HDF__ (Иерархический формат данных) — название формата файлов, разработанного для хранения большого объема цифровой информации. 
* Поддерживается некоммерческой организацией HDF Group.
* Библиотеки для работы с форматом и связанные с ним утилиты доступны для использования под свободной лицензией.
* Имеются библиотеки для работы с HDF для большинства языков программирования, активно использующихся для обработки числовых данных (в частности: Java, Matlab, Scilab, Octave, Mathematica, IDL, Python, R и Julia)
* Существуют отдельные инструменты для работы с HDF, например: https://support.hdfgroup.org/products/java/hdfview/

На данный момент в ходу следующие варианты HDF:
* __HDF4__ — старая версия формата, однако все еще активно поддерживаемая HDF Group.
    * Формат поддерживает различные модели данных, включая многомерные массивы, растровые изображения и таблицы.
    * Использует 32-битные целые числа, поэтому имеет проблемы с хранением больших объёмов инфрмации (более нескольких гигабайт).
* __HDF5__ — современная версия формата. Содержит иерархию из двух основных типов объектов:
    * Datasets — наборы данных, многомерные массивы объектов одного типа
    * Groups — группы, являются контейнерами для наборов данных и других групп
    * Содержимое файлов HDF5 организовано подобно иерархической файловой системе, и для доступа к данным применяются пути, сходные с POSIX-синтаксисом, например, `/path/to/resource`.
    * Метаданные хранятся в виде набора именованных атрибутов объектов.
    
<center>         
    <img src="./img/hdf5.png" alt="Логика организации данных в HDF5" style="width: 500px;"/>
    <b>Логика организации данных в HDF5</b>
</center>    

Популярный модуль для работы с HDF5 на Python: `h5py`.
* Документация по `h5py`: https://docs.h5py.org/en/stable/
* Рекомендованное руководство для изучения работы с HDF5 в python: https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/
* Перевод руководства: https://habr.com/ru/company/otus/blog/416309/

In [141]:
# import numpy as np

# модуль для работы с HDF5:
import h5py

import time
import os

In [142]:
len(a1), a1[:10], a1[-10:]

(100,
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]))

In [143]:
with h5py.File('data1.hdf5', 'w') as f:
    dset = f.create_dataset("a1", data=a1)

In [144]:
os.path.getsize('data1.hdf5')

2448

In [145]:
with h5py.File('data1.hdf5', 'r') as f:
    a1_l = f['a1']
    print(len(a1), a1[:10], a1[-10:])

100 [0 1 2 3 4 5 6 7 8 9] [90 91 92 93 94 95 96 97 98 99]


In [146]:
# hdf5 позволяет обойти сохраненные в файле датасеты:
with h5py.File('data1.hdf5', 'r') as f:
    for key in f.keys():
        print(key)

a1


In [148]:
f = h5py.File('data1.hdf5', 'r')
a1_l = f['a1']

In [149]:
# пока файл не закрыт, мы можем пользоваться датасетом, в т.ч. с использованием срезов:
a1_l[:10], a1_l[-10:]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]))

In [150]:
# тип a1:
type(a1_l)

h5py._hl.dataset.Dataset

In [151]:
f.close()

In [63]:
# ошибка! 
# a1_l[:10]

In [152]:
with h5py.File('data1.hdf5', 'r') as f:
    a1_l = f['a1']
    data = a1_l[:10]

In [153]:
 a1_l[1]

ValueError: Not a dataset (not a dataset)

In [154]:
# Ошибки нет, т.к. при выполнении среза a1_l[:10] данные были прочитаны и сохранены в data: 
data[1]

1

In [155]:
type(data)

numpy.ndarray

Запись в HDF5

In [157]:
a_rnd1 = np.random.randn(100)

with h5py.File('random1.hdf5', 'w') as f:
    dset = f.create_dataset("default", (1000,))
    dset[10:20] = a_rnd1[50:60] # записываем только часть датасета!

In [158]:
with h5py.File('random1.hdf5', 'r') as f:
    print(f['default'][:30])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.7179446  -0.96145785
 -1.0227864  -0.12980658  0.6067258  -1.1655966   0.9502432  -1.4991108
 -1.0368686   0.6302976   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [159]:
a_rnd2 = np.random.randn(1000)

In [160]:
with h5py.File('random2.hdf5', 'w') as f:
    dset = f.create_dataset("default", (1000,))
    dset = a_rnd2 # ошибка!

In [161]:
with h5py.File('random2.hdf5', 'r') as f:
    print(f['default'][:10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [162]:
with h5py.File('random2.hdf5', 'w') as f:
    dset = f.create_dataset("default", (1000,))
    dset[:] = a_rnd2 # копирование производится при обходе среза!

In [163]:
with h5py.File('random2.hdf5', 'r') as f:
    print(f['default'][:10])

[ 0.3328062  -1.1835508   1.0105177  -0.26330495  0.11441141 -0.41643113
 -0.7413635   0.1412695  -2.261247   -0.64286685]


Типизация датасетов в HDF5
* документация: https://docs.h5py.org/en/latest/faq.html

In [164]:
with h5py.File('several_datasets.hdf5', 'w') as f:
    # объявление нескольких типизированных датасетов в одном файле:
    dset_int_1 = f.create_dataset('integers', (10, ), dtype='i1')
    dset_int_8 = f.create_dataset('integers8', (10, ), dtype='i8')
    dset_complex = f.create_dataset('complex', (10, ), dtype='c16')

    # помещение данных в датасеты:
    dset_int_1[0] = 1200
    dset_int_8[0] = 1200.1
    dset_complex[0] = 3 + 4j

In [76]:
arr = np.random.randn(100000)

f = h5py.File('integer_1.hdf5', 'w')
d = f.create_dataset('dataset', (100000,), dtype='i1')
d[:] = arr
print(type(d[0]))
f.close()
print(os.path.getsize('integer_1.hdf5'))

f = h5py.File('integer_8.hdf5', 'w')
d = f.create_dataset('dataset', (100000,), dtype='i8')
d[:] = arr
print(type(d[0]))
f.close()
print(os.path.getsize('integer_8.hdf5'))

f = h5py.File('float_2.hdf5', 'w')
d = f.create_dataset('dataset', (100000,), dtype='f2')
d[:] = arr
print(type(d[0]))
f.close()
print(os.path.getsize('float_2.hdf5'))

<class 'numpy.int8'>
102048
<class 'numpy.int64'>
802048
<class 'numpy.float16'>
202048


Создание сжатых файлов:

In [165]:
with h5py.File('integer_1_compr.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (100000,), dtype='i1', 
                         compression="gzip", compression_opts=9)
    d[:] = arr
print(os.path.getsize('integer_1.hdf5'))
print(os.path.getsize('integer_1_compr.hdf5'))

with h5py.File('integer_8_compr.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (100000,), dtype='i8', compression="gzip", compression_opts=9)
    d[:] = arr
print(os.path.getsize('integer_8.hdf5'))    
print(os.path.getsize('integer_8_compr.hdf5'))    

with h5py.File('float_2_compr.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (100000,), dtype='f2', compression="gzip", compression_opts=9)
    d[:] = arr
print(os.path.getsize('float_2_compr.hdf5'))    
print(os.path.getsize('float_2.hdf5'))    

102048
28009
802048
43402
188337
202048


Изменение размера датасета

In [166]:
with h5py.File('resize_dataset.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (100, ),  maxshape=(500, ))
    d[:100] = np.random.randn(100)
    d.resize((200,))
    d[100:200] = np.random.randn(100)

with h5py.File('resize_dataset.hdf5', 'r') as f:
    dset = f['dataset']
    print(dset[99])
    print(dset[199])

2.7788746
-0.35447088


In [167]:
# изменение размера датасета в уже существовавшем файле:
with h5py.File('resize_dataset.hdf5', 'a') as f:
    dset = f['dataset']
    dset.resize((300,))
    dset[:200] = 0
    dset[200:300] = np.random.randn(100)

with h5py.File('resize_dataset.hdf5', 'r') as f:
    dset = f['dataset']
    print(dset[99])
    print(dset[199])
    print(dset[299])

0.0
0.0
-1.6963162


Сохранение данных блоками (Chunks)

Чтобы оптимизировать хранение данных, их можно хранить блоками (chunk). 
* Каждый блок (chunk) будет смежным на жестком диске и будет храниться единым фрагментом, т.е. весь фрагмент будет записан сразу. 
* Когда вы читаете блок, произойдет то же самое - он будут загружен целиком. Чтобы создать кусочный набор данных.

In [168]:
with h5py.File('chunked_dataset.hdf5', 'a') as f:
    dset1 = f.create_dataset("chunked2", (1000, 1000), 
                             chunks=(100, 100))
    dset2 = f.create_dataset("autochunk2", (1000, 1000), 
                             chunks=True)

RuntimeError: Unable to create link (name already exists)

Организация данных группами (Groups)

Группы (Groups) позволяют организовать информацию в файле. Наборы данных могут быть размещены внутри групп, которые ведут себя аналогично тому, как работают каталоги в файловой системе. Сначала мы можем создать группу, а затем добавить в нее набор данных.

In [169]:
with h5py.File('groups.hdf5', 'w') as f:
    g = f.create_group('Base_Group')
    gg = g.create_group('Sub_Group')

    d = g.create_dataset('default', data=arr)
    dd = gg.create_dataset('default', data=arr)

In [170]:
with h5py.File('groups.hdf5', 'r') as f:
    d = f['Base_Group/default']
    dd = f['Base_Group/Sub_Group/default']
    print(d[1])
    print(dd[1])

0.3189208475958385
0.3189208475958385


In [171]:
with h5py.File('groups.hdf5', 'r') as f:
    for k in f.keys():
        print(k)

Base_Group


In [172]:
# способ обойти 
def get_all(name):
    print(name)

with h5py.File('groups.hdf5', 'r') as f:
    f.visit(get_all)

Base_Group
Base_Group/Sub_Group
Base_Group/Sub_Group/default
Base_Group/default


In [173]:
def get_all(name):
    if 'Sub_Group' in name:
        return name

with h5py.File('groups.hdf5', 'r') as f:
    g = f.visit(get_all)
    print(g)

Base_Group/Sub_Group


In [174]:
with h5py.File('groups.hdf5', 'r') as f:
    g_name = f.visit(get_all)
    group = f[g_name]
    for k, ds in group.items():
        print(k)
        print(ds[0])
#     print(group[0])

default
-0.4119813716273497


In [175]:
def get_objects(name, obj):
    if 'Sub_Group' in name:
        return obj

with h5py.File('groups.hdf5', 'r') as f:
    group = f.visititems(get_objects)
    data = group['default']
    print('First data element: {}'.format(data[0]))

First data element: -0.4119813716273497


__Хранение метаданных в HDF5__

Одной из важных возможностей HDF5 является возможность хранения метаданных, прикрепленных к любой группе или набору данных. Метаданные позволяют понять, например, источник данных, каковы параметры, используемые для измерения или моделирования, и т.д. Метаданные делают файл самоописательным.

In [176]:
with h5py.File('groups.hdf5', 'w') as f:
    g = f.create_group('Base_Group')
    d = g.create_dataset('default', data=arr)

    g.attrs['Date'] = time.time()
    g.attrs['User'] = 'Me'

    d.attrs['OS'] = os.name

    for k in g.attrs.keys():
        print('{} => {}'.format(k, g.attrs[k]))

    for j in d.attrs.keys():
        print('{} => {}'.format(j, d.attrs[j]))

Date => 1633428611.4651322
User => Me
OS => nt


In [212]:
with h5py.File('groups.hdf5', 'w') as f:
    g = f.create_group('Base_Group')
    d = g.create_dataset('default', data=arr)

    metadata = {'Date': time.time(),
      'User': 'Me',
      'OS': os.name,}

    f.attrs.update(metadata)

    for m in f.attrs.keys():
        print('{} => {}'.format(m, f.attrs[m]))

Date => 1601647108.3349116
OS => nt
User => Me


__Использование HDF5 для работы с Pandas__

```python
DataFrame.to_hdf(path_or_buf, key, mode='a', complevel=None, complib=None, append=False, format=None, index=True, min_itemsize=None, nan_rep=None, dropna=None, data_columns=None, errors='strict', encoding='UTF-8')
```
Документация: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html#pandas.DataFrame.to_hdf

Некоторые параметры:

* mode : {'a', 'w', 'r+'}, default 'a' . 
* complevel : {0-9}, optional, Specifies a compression level for data. A value of 0 disables compression.
* complib : {'zlib', 'lzo', 'bzip2', 'blosc'}, default 'zlib'
* format : {'fixed', 'table', None}, default 'fixed'
            Possible values:
    * 'fixed': Fixed format. Fast writing/reading. Not-appendable, nor searchable.
    * 'table': Table format. Write as a PyTables Table structure which may perform worse but allow more flexible operations               like searching / selecting subsets of the data.
    * If None, pd.get_option('io.hdf.default_format') is checked,              followed by fallback to "fixed"
* min_itemsize : dict or int, optional.  Map column names to minimum string sizes for columns.
* nan_rep : Any, optional. How to represent null values as str. 
* data_columns : list of columns or True, optional

```python
DataFrame.to_hdf(path_or_buf, key, mode='a', complevel=None, complib=None, append=False, format=None, index=True, min_itemsize=None, nan_rep=None, dropna=None, data_columns=None, errors='strict', encoding='UTF-8')
```

Документация: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_hdf.html

In [177]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]},
                  index=['a', 'b', 'c'])
df1

,A,B
a,1,4
b,2,5
c,3,6


In [178]:
df1.to_hdf('pandas.hdf5', key='df', mode='w')

In [179]:
# как все хранится внутри:
with h5py.File('pandas.hdf5', 'r') as f:
    data = f['df']
    print(data)
    for k, ds in data.items():
        print(k)
        print(ds[:])    

<HDF5 group "/df" (4 members)>
axis0
[b'A' b'B']
axis1
[b'a' b'b' b'c']
block0_items
[b'A' b'B']
block0_values
[[1 4]
 [2 5]
 [3 6]]


In [180]:
df_l = pd.read_hdf('data.h5', 'df')
df_l

,A,B
a,1,4
b,2,5
c,3,6


In [181]:
# Добавляем серию в тот же файл:
s = pd.Series([1, 2, 3, 4])
s.to_hdf('data.h5', key='s')

In [182]:
pd.read_hdf('data.h5', 's')

0    1
1    2
2    3
3    4
dtype: int64

# Apache Parquet <a class="anchor" id="parquet"></a>
* [к оглавлению](#разделы)

Apache Parquet хранит данные способом, при котором значения в каждом столбце физически хранятся в смежных ячейках памяти. Благодаря колоночно-ориентированному хранилищу Apache Parquet обеспечивает следующие преимущества:

* Сжатие по столбцам выполняется эффективно и экономит место для хранения данных. 
    * т.к. могут применяться методы сжатия, специфичные для определенного типа, поскольку значения столбцов, как правило, имеют один и тот же тип;
    * к разным столбцам можно применять разные методы кодирования.    
* Можно читать и десериализовывать только те столбцы, которые необходимы.
    * Запросы, которые извлекают значения из определенного столбца, не должны считывать данные всей строки, что повышает производительность.
    

<center>         
    <img src="./img/column.png" alt="Принцип колоночного хранилища" style="width: 700px;"/>
    <b>Принцип колоночного хранилища</b>
</center>

__Сравнение с другими популярными форматами__

<center>         
    <img src="./img/parquet_vs.png" alt="CSV vs JSON vs Parquet" style="width: 600px;"/>
    <b>CSV vs JSON vs Parquet</b>
</center>

* `*` CSV is splittable when it is a raw, uncompressed file or using a splittable compression format such as BZIP2 or LZO (note: LZO needs to be indexed to be splittable!)

* `**` JSON has the same conditions about splittability when compressed as CSV with one extra difference. When “wholeFile” option is set to true (re: SPARK-18352), JSON is NOT splittable.

Основные преимущества каждого из форматов:
* CSV should generally be the fastest to write
    * CSV is the defacto standard of a lot of data and for fair reasons; 
    * it’s (relatively) easy to comprehend for both users and computers and made more accessible via Microsoft Excel.
* JSON the easiest for a human to understand
    * JSON is the standard for communicating on the web. 
    * APIs and websites are constantly communicating using JSON because of its usability properties such as well-defined schemas.
* Parquet the fastest to read.
    * Parquet is optimized for the __Write Once Read Many (WORM)__ paradigm. 
    * It’s slow to write, but incredibly fast to read, especially when you’re only accessing a subset of the total columns. 
    * For use cases __requiring operating on entire rows of data__, a format like __CSV, JSON or even AVRO should be used__.

__Сравнение форматов для хранения больших данных__

<center>         
    <img src="./img/bd_formats.png" alt="Сравнение форматов для хранения больших данных" style="width: 500px;"/>
    <b>Сравнение форматов для хранения больших данных</b>
</center>

__Avro__ - это ориентированная на строки структура удаленного вызова процедур и сериализации данных, разработанная в рамках проекта Apache Hadoop. Он использует JSON для определения типов данных и протоколов и сериализует данные в компактном двоичном формате. Его основное применение - в Apache Hadoop, где он может предоставлять как формат сериализации для постоянных данных, так и формат проводной связи для связи между узлами Hadoop и от клиентских программ к службам Hadoop.

__Avro__ is a row-oriented remote procedure call and data serialization framework developed within Apache's Hadoop project. It uses JSON for defining data types and protocols, and serializes data in a compact binary format. Its primary use is in Apache Hadoop, where it can provide both a serialization format for persistent data, and a wire format for communication between Hadoop nodes, and from client programs to the Hadoop services. 


<center>         
    <img src="./img/size_comparison.png" alt="Сравнение форматов для хранения больших данных" style="width: 500px;"/>
    <b>Сравнение форматов для хранения больших данных</b>
</center>

<center>         
    <img src="./img/orc_file.png" alt="Сравнение форматов для хранения больших данных" style="width: 500px;"/>
    <b>Организация хранения в файле формата ORC</b>
</center>


In [219]:
import sys

__Apache Arrow__

__Apache Arrow__ - это не зависящая от языка программная среда для разработки приложений анализа данных, которые __обрабатывают колоночно-ориентированные данные__. Он содержит стандартизированный формат памяти с ориентацией на столбцы, который может представлять плоские и иерархические данные для эффективных аналитических операций на современном аппаратном обеспечении CPU и GPU. Это снижает или устраняет факторы, ограничивающие возможность работы с большими наборами данных, такие как стоимость, непостоянство хранения или физические ограничения динамической памяти с произвольным доступом.

PyArrow это API для Apache Arrow на Python. Реализация ядра Apache Arrow выполнена на C++, а PyArrow реализует интерфейс к этому ядру, реализуя принцип "Python as a 'glue language' ". PyArrow поддерживает полноценную интеграцию с:
* NumPy
* Pandas
* встроенными объектами Python

Работу с файлами parquet будем организовывать с помощью __PyArrow__ .
* документация: https://enpiar.com/arrow-site/docs/python/index.html
* работа с Pandas: https://enpiar.com/arrow-site/docs/python/parquet.html 

Установка и импорт модуля pyarrow:

In [5]:
!conda install --yes --prefix {sys.prefix} pyarrow arrow-cpp parquet-cpp -c conda-forge

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3\envs\pyTorch_1_6

  added / updated specs:
    - arrow-cpp
    - parquet-cpp
    - pyarrow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    arrow-cpp-0.15.1           |   py37h47fa567_6         2.9 MB
    boost-cpp-1.68.0           |    h6a4c333_1000        31.1 MB  conda-forge
    brotli-1.0.9               |       ha925a31_0         885 KB  conda-forge
    c-ares-1.16.1              |       he774522_3         106 KB  conda-forge
    double-conversion-3.1.5    |       h6538335_2         544 KB  conda-forge
    gflags-2.2.2               |    ha925a31_1004          80 KB  conda-forge
    glog-0.4.0                 |       h0174b99_3          83 KB  conda-forge
    grpc-cpp-1.26.0            |       h351948d_0         4.8 MB
    libboost-1.67.0            |       hd9e427e_4      



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.8.5

Please update conda by running

    $ conda update -n base -c defaults conda




In [3]:
# альтернативный способ:
# %conda install pyarrow arrow-cpp parquet-cpp -c conda-forge


Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ўлЇ®«­Ёвм гЄ § ­­го Їа®Ја ¬¬г.


In [70]:
import pyarrow.parquet as pq

In [71]:
df1

,A,B
a,1,4
b,2,5
c,3,6


Запись Pandas DataFrame в parquet
```python
DataFrame.to_parquet(**kwargs)
```
Write a DataFrame to the binary parquet format.

This function writes the dataframe as a parquet file. You can choose different parquet backends, and have the option of compression.

__Parameters__
* path: str or file-like object
* engine: {‘auto’, ‘pyarrow’, ‘fastparquet’}, default ‘auto’
* compression: {‘snappy’, ‘gzip’, ‘brotli’, None}, default ‘snappy’
* index: bool, default None
    If True, include the dataframe’s index(es) in the file output.
* partition_cols: list, optional, default None

In [72]:
df1.to_parquet('df1.parquet.gzip', compression='gzip')

Загрузка Pandas DataFrame данных из parquet:

```python
pandas.read_parquet(path, engine='auto', columns=None, **kwargs)
```

Load a parquet object from the file path, returning a DataFrame.

* path : str, path object or file-like object
        Any valid string path is acceptable. The string could be a URL. Valid
        URL schemes include http, ftp, s3, and file. For file URLs, a host is
        expected. A local file could be:
        ``file://localhost/path/to/table.parquet``.
        A file URL can also be a path to a directory that contains multiple
        partitioned parquet files. Both pyarrow and fastparquet support
        paths to directories as well as file URLs. A directory path could be:
        ``file://localhost/path/to/tables`` or ``s3://bucket/partition_dir``
        If you want to pass in a path object, pandas accepts any
        ``os.PathLike``.
        By file-like object, we refer to objects with a ``read()`` method,
        such as a file handler (e.g. via builtin ``open`` function)
        or ``StringIO``.
* engine : {'auto', 'pyarrow', 'fastparquet'}, default 'auto'
        Parquet library to use. If 'auto', then the option
        ``io.parquet.engine`` is used. The default ``io.parquet.engine``
        behavior is to try 'pyarrow', falling back to 'fastparquet' if
        'pyarrow' is unavailable.
* columns : list, default=None
        If not None, only these columns will be read from the file.

In [73]:
df1_l = pd.read_parquet('df1.parquet.gzip')

df1_l

,A,B
a,1,4
b,2,5
c,3,6


Разделение содержимого parquet на файлы (Partitioned Datasets):

* https://enpiar.com/arrow-site/docs/python/parquet.html#partitioned-datasets-multiple-files

In [74]:
df1.to_parquet('df1_prt.parquet.gzip', partition_cols = ['A'], compression='gzip')

In [75]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

In [76]:
# структура директории:
list_files('./df1_prt.parquet.gzip')

df1_prt.parquet.gzip/
    A=1/
        96cda9032f074ee0a299cf39eb9f7cb6.parquet
        b6db53d9a02047c6926a94d7097c10eb.parquet
        f6f7f03e22ab44ff843ca3b38f97d3d8.parquet
    A=2/
        1362e6fefaeb4fa3aa83bc865edc4e39.parquet
        4db82abfce4b48718caf741b5228c7f7.parquet
        5b52e72a86754b629a32e2b08f4d59e5.parquet
    A=3/
        40df716c3ff5416cade748f6c58941a2.parquet
        6023123d6df243d6a0b9befc9c37e887.parquet
        7c32c070003c4594b4e03bbbee36e980.parquet


In [77]:
df1_lp = pd.read_parquet('df1_prt.parquet.gzip')
df1_lp

,B,__index_level_0__,A
0,4,a,1
1,4,a,1
2,4,a,1
3,5,b,2
4,5,b,2
5,5,b,2
6,6,c,3
7,6,c,3
8,6,c,3
